In [1]:
import pandas as pd
import numpy as np
import preprocessingFuncts as pp
from sklearn import neighbors
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt

In [2]:
#all base datas

def readData(paths, base='', categories=[]):
  arr = []
  itemData = pp.readItemData()
  userData = pp.readUserData()
  for path in paths:
    ratingData = pp.readRatingData(path)
    data = 0
    if base == "user":
      data = pp.specifyByUserData(userData, ratingData, categories)
    elif base == "item":
      data = pp.specifyByItemData(itemData, ratingData, categories)
    else:
      data = ratingData
    data = data.dropna()
    y = data.loc[:, 'rating']
    x = data.copy()
    x = x.drop(['rating'], axis='columns')
    arr.append({'x': x, 'y': y, 'name': path[8:10], 'type': path[-4:]})
  return arr

In [3]:
from models import ItemBasedCF

#Creating the model & training it
IBCF = ItemBasedCF()
IBCF.fit('ml-100k\\ua.base')

In [4]:
#Viewing the data matrix
IBCF.dataMatrix

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Making ratings prediction

testData = pp.readRatingData('ml-100k\\ua.test')
testX, testY =  testData.loc[:,['user_id','item_id']],testData.loc[:,'rating']

y = IBCF.predict(testX)
y

[5,
 4,
 5,
 3,
 4,
 4,
 5,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 0,
 0,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 0,
 3,
 3,
 3,
 3,
 0,
 3,
 4,
 0,
 0,
 0,
 3,
 4,
 3,
 3,
 5,
 3,
 3,
 4,
 4,
 5,
 5,
 4,
 4,
 5,
 4,
 4,
 5,
 4,
 5,
 5,
 4,
 4,
 5,
 4,
 4,
 3,
 4,
 3,
 0,
 4,
 0,
 4,
 4,
 4,
 5,
 5,
 4,
 4,
 5,
 0,
 4,
 0,
 4,
 4,
 5,
 4,
 5,
 5,
 5,
 5,
 3,
 0,
 4,
 3,
 4,
 4,
 0,
 0,
 4,
 4,
 5,
 5,
 4,
 4,
 5,
 4,
 4,
 5,
 4,
 5,
 3,
 2,
 3,
 4,
 3,
 3,
 5,
 0,
 4,
 0,
 4,
 4,
 4,
 5,
 3,
 4,
 5,
 4,
 5,
 5,
 2,
 3,
 3,
 3,
 4,
 3,
 1,
 2,
 4,
 2,
 4,
 5,
 4,
 4,
 4,
 4,
 5,
 5,
 4,
 5,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 4,
 4,
 0,
 3,
 4,
 5,
 4,
 4,
 3,
 0,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 2,
 4,
 4,
 0,
 3,
 3,
 3,
 3,
 0,
 0,
 4,
 5,
 0,
 3,
 3,
 5,
 3,
 2,
 0,
 0,
 2,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [6]:
# TODO - accuracy

In [7]:
# TODO - confusion matrix